In [3]:
import re
from collections import defaultdict
import pandas as pd

def read_conllu_file(conllu_file):
    sentences = defaultdict(list)
    current_id = None

    with open(conllu_file, 'r', encoding='utf-8') as infile:
        for line in infile:
            line = line.strip()
            if line.startswith("# sent_id = "):
                current_id = line.replace("# sent_id = ", "")
                # Remover o primeiro "dante_01_" se duplicado
                if current_id.count("dante_01_") > 1:
                    current_id = current_id.replace("dante_01_", "", 1)
            elif line.startswith("#"):
                continue
            elif line == "":
                current_id = None
            else:
                if current_id is not None:
                    parts = line.split("\t")
                    if len(parts) > 1:
                        sentences[current_id].append(parts[1])

    return sentences

def compare_tokenizations(file1, file2, report_file):
    tokens1 = read_conllu_file(file1)
    tokens2 = read_conllu_file(file2)
    
    differences = []

    all_ids = set(tokens1.keys()).union(set(tokens2.keys()))

    for id in all_ids:
        tokens_seq1 = tokens1.get(id, [])
        tokens_seq2 = tokens2.get(id, [])
        
        if tokens_seq1 != tokens_seq2:
            differences.append({
                'ID': id,
                'File1 Tokens': ' '.join(tokens_seq1),
                'File2 Tokens': ' '.join(tokens_seq2)
            })

    df = pd.DataFrame(differences)
    df.to_csv(report_file, index=False)
    print(f"Relatório de diferenças salvo em {report_file}")

# Caminhos dos arquivos conllu e do relatório
conllu_file1 = r'C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1_reconstruido.conllu'  # Substitua pelo caminho do seu arquivo
conllu_file2 = r'C:\Users\Marcos\Desktop\tic\bases\DANTEStocksV2.1.conllu'  # Substitua pelo caminho do seu arquivo
report_file = r'comparacao.csv'

# Comparar as tokenizações e gerar o relatório
compare_tokenizations(conllu_file1, conllu_file2, report_file)


Relatório de diferenças salvo em comparacao.csv


In [5]:
import pandas as pd
from jinja2 import Template
from difflib import SequenceMatcher

def find_differences(seq1, seq2):
    """Encontra as diferenças entre duas sequências de tokens"""
    matcher = SequenceMatcher(None, seq1, seq2)
    diferentes_seq1 = set()
    diferentes_seq2 = set()
    
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag != 'equal':  # se não são iguais, marca os tokens como diferentes
            diferentes_seq1.update(range(i1, i2))
            diferentes_seq2.update(range(j1, j2))
            
    return diferentes_seq1, diferentes_seq2

def create_comparison_report(csv_file, html_report_file):
    # Carregar o CSV gerado previamente
    df = pd.read_csv(csv_file)
    
    # Renomear as colunas
    df = df.rename(columns={'File1 Tokens': 'Dante Reconstruido', 'File2 Tokens': 'Dante Original'})
    df['Dante Reconstruido'] = df['Dante Reconstruido'].fillna('').astype(str)
    df['Dante Original'] = df['Dante Original'].fillna('').astype(str)

    # Template HTML permanece o mesmo...
    html_template = """
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Relatório de Comparação de Tokenização</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                line-height: 1.6;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }
            th, td {
                padding: 12px;
                border: 1px solid #ddd;
                text-align: left;
                vertical-align: top;
            }
            th {
                background-color: #f4f4f4;
                font-weight: bold;
            }
            tr:nth-child(even) {
                background-color: #f9f9f9;
            }
            .diff {
                background-color: #ffcccc;
                padding: 2px 4px;
                border-radius: 3px;
            }
            .token {
                margin: 0 2px;
            }
            h1 {
                color: #333;
                border-bottom: 2px solid #333;
                padding-bottom: 10px;
            }
        </style>
    </head>
    <body>
        <h1>Relatório de Comparação de Tokenização</h1>
        <table>
            <thead>
                <tr>
                    <th>ID</th>
                    <th>Dante Reconstruido</th>
                    <th>Dante Original</th>
                </tr>
            </thead>
            <tbody>
                {% for row in rows %}
                <tr>
                    <td>{{ row.ID }}</td>
                    <td>{{ row.Dante_Reconstruido | safe }}</td>
                    <td>{{ row.Dante_Original | safe }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </body>
    </html>
    """

    # Preparar os dados para o template com comparação melhorada
    rows = []
    for _, row in df.iterrows():
        # Dividir os tokens
        dante_reconstruido_tokens = row['Dante Reconstruido'].split()
        dante_original_tokens = row['Dante Original'].split()

        # Encontrar as diferenças usando SequenceMatcher
        diferentes_reconstruido, diferentes_original = find_differences(
            dante_reconstruido_tokens, 
            dante_original_tokens
        )

        # Marcar apenas as diferenças
        dante_reconstruido_tokens_str = " ".join([
            f"<span class='diff token'>{token}</span>" if i in diferentes_reconstruido 
            else f"<span class='token'>{token}</span>" 
            for i, token in enumerate(dante_reconstruido_tokens)
        ])
        
        dante_original_tokens_str = " ".join([
            f"<span class='diff token'>{token}</span>" if i in diferentes_original 
            else f"<span class='token'>{token}</span>" 
            for i, token in enumerate(dante_original_tokens)
        ])

        rows.append({
            'ID': row['ID'],
            'Dante_Reconstruido': dante_reconstruido_tokens_str,
            'Dante_Original': dante_original_tokens_str
        })

    # Renderizar o template com os dados
    template = Template(html_template)
    html_content = template.render(rows=rows)

    # Salvar o relatório HTML
    with open(html_report_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    print(f"Relatório HTML salvo em {html_report_file}")

# Caminhos do arquivo CSV e do relatório HTML
csv_file = r'comparacao.csv'
html_report_file = r'comparacao.html'

# Criar o relatório HTML
create_comparison_report(csv_file, html_report_file)

Relatório HTML salvo em comparacao.html
